https://www.ft.dk/aktuelt/webtv/video/20222/salen/5.aspx?from=28-11-2022&to=19-12-2022&selectedMeetingType=Salen&committee=&as=1#player


In [88]:
import requests # Downloading webpages
from bs4 import BeautifulSoup # Extracting data from html files
import json
import pandas as pd
import re

In [89]:
url = "https://www.ft.dk/aktuelt/webtv/video/20222/salen/5.aspx?from=28-11-2022&to=19-12-2022&selectedMeetingType=Salen&committee=&as=1#player"
req = requests.get(url) # get the page

In [90]:
soup = BeautifulSoup(req.text, 'html.parser')

# Initialize lists to store the extracted data
names = []
texts = []
numbers = []

# Find all divs with class 'video-item-referat'
for div in soup.find_all("div", class_="video-item-referat"):
    # Extract name
    name_div = div.find("div", class_="name")
    name = name_div.get_text(strip=True)

    # Extract texts from all <p> with class 'Tekst' and 'TekstIndryk'
    text_elements = div.find_all("p", class_=["Tekst", "TekstIndryk"])
    combined_text = ' '.join(p.get_text(strip=True) for p in text_elements)

    # Extract number (from id of 'video-item-content')
    content_div = div.find("div", class_="video-item-content")
    number = content_div['id'].split('_')[-1] if content_div and 'id' in content_div.attrs else 'N/A'

    # Append to lists
    names.append(name)
    texts.append(combined_text)
    numbers.append(number)

# Create DataFrame
df = pd.DataFrame({
    'Name': names,
    'Text': texts,
    'Number': numbers
})

# Remove rows that contain 'N/A'
df = df[df.Number != 'N/A']

In [91]:
df.head()

,Name,Text,Number
0,Formanden Søren Gade,Der er som bekendt ingen minister på dette omr...,912053
1,Jens Joel (S),"Tak for det, formand. Og ja, det er rigtigt, a...",912064
2,Formanden Søren Gade,"Tak for det. Der er ikke nogen, der har bedt o...",912075
3,Thomas Danielsen (V),"Tak for det, formand. Venstre bifalder EU-refo...",912078
4,Formanden Søren Gade,"Tak for det. Der er ikke nogen, der har bedt o...",912079


In [92]:
script_tag = soup.find('script', text=re.compile('SpeachItem'))
# Extract all instances of SpeachItem
speach_items = re.findall(r'new SpeachItem\(.*\)', script_tag.string)
# Remove 'new SpeachItem'
speach_items = [re.sub(r'new SpeachItem', '', item) for item in speach_items]

/var/folders/fy/dq076l2x6h193qvwcs1f0qm00000gn/T/ipykernel_32680/1089534222.py:1: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  script_tag = soup.find('script', text=re.compile('SpeachItem'))


In [93]:
# loop over speach_items and extract the data
start_times = []
end_times = []
numbers = []

for item in speach_items:
    start_times.append(item[19:27])
    end_times.append(item[41:49])
    numbers.append(item[58:-2])

# Create DataFrame
time_stamps = pd.DataFrame({
    'Start': start_times,
    'End': end_times,
    'Number': numbers
})


In [94]:
time_stamps.head()

,Start,End,Number
0,10:00:51,10:01:28,912053
1,10:01:28,10:06:41,912064
2,10:06:41,10:06:49,912075
3,10:06:49,10:07:25,912078
4,10:07:25,10:07:36,912079


In [95]:
#combine the two dataframes on the number column
df = pd.merge(df, time_stamps, on='Number')
# drop the number column
df = df.drop(columns=['Number'])

In [96]:
df

,Name,Text,Start,End
0,Formanden Søren Gade,Der er som bekendt ingen minister på dette omr...,10:00:51,10:01:28
1,Jens Joel (S),"Tak for det, formand. Og ja, det er rigtigt, a...",10:01:28,10:06:41
2,Formanden Søren Gade,"Tak for det. Der er ikke nogen, der har bedt o...",10:06:41,10:06:49
3,Thomas Danielsen (V),"Tak for det, formand. Venstre bifalder EU-refo...",10:06:49,10:07:25
4,Formanden Søren Gade,"Tak for det. Der er ikke nogen, der har bedt o...",10:07:25,10:07:36
5,Henrik Frandsen (M),"Mange tak, hr. formand. Jeg vil gerne starte m...",10:07:36,10:09:15
6,Formanden Søren Gade,"Tak for det. Der er ikke nogen, der har bedt o...",10:09:15,10:09:30
7,Karina Lorentzen Dehnhardt (SF),"Det her er jo et af de forslag, som der, efter...",10:09:30,10:10:09
8,Formanden Søren Gade,"Tak for det. Der er ikke nogen, der har bedt o...",10:10:09,10:10:27
9,Peter Skaarup (DD),Jeg skal også på Danmarksdemokraternes vegne t...,10:10:27,10:11:26
